<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import psycopg2 as pg
import pandas as pd
import numpy as np
import pandas.io.sql as psql
import datetime
from dateutil.relativedelta import relativedelta
import time
import sys
sys.path.append('../src')
from datahub import PostgreSQLBase, ToDatahubWriter
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(name)s : %(levelname)s : %(message)s')

consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(formatter)
logger.addHandler(consoleHandler)

import auxiliary as aux
reload (aux)

import features
reload(features)
import revenue
reload(revenue)

postgres = PostgreSQLBase()
df_proc = aux.DataFrameProcessor()
loc_ids = aux.City_Country()

ImportError: DLL load failed: The application has failed to start because its side-by-side configuration is incorrect. Please see the application event log or use the command-line sxstrace.exe tool for more detail.

In [2]:
from IPython.display import display

In [3]:
import plotly.plotly as py
import plotly


In [4]:
%%time
df = postgres.get_data('active_positions_per_date_dataset_snaps.sql', param='2019-03-31', param_type='day')

Wall time: 10min 33s


In [5]:
df.shape

(647059, 61)

In [ ]:
"""select position_id, staffing_status
from wpm_head.epm_staf_m_positions
where position_id in {}"""

In [9]:
pos = postgres.get_data('test_query.sql', param=tuple(df.position_id.unique().astype(str)), param_type='day')

In [14]:
df = pd.merge(
    df, pos.set_index('position_id')[['staffing_status']].rename(column={'staffing_status' : 'staffing_status_final'}),
    how='left', left_on = 'position_id', right_index=True
)

In [15]:
df.shape

(647059, 65)

In [20]:
df = df_proc.datetime_cols(df, ['date', 'timesheet_date'])


df['cur_month'] = df.date.map(lambda d: datetime.datetime.strftime(d, '%Y-%m-%d')[:7])
df['next_1_month'] = df.date.map(
    lambda d: datetime.datetime.strftime(d + relativedelta(months=1), '%Y-%m-%d')[:7])
df['next_2_month'] = df.date.map(
    lambda d: datetime.datetime.strftime(d + relativedelta(months=2), '%Y-%m-%d')[:7])

df['report_month'] = df[
    df.timesheet_date.notnull()
].timesheet_date.map(lambda d: datetime.datetime.strftime(d, '%Y-%m-%d')[:7])

2019-07-26 14:57:30,864 - model.auxiliary : DEBUG : datetime columns process started
2019-07-26 14:57:30,927 - model.auxiliary : DEBUG : date
2019-07-26 14:57:30,941 - model.auxiliary : DEBUG : timesheet_date


In [21]:
df['next_plus_month'] = np.where(
    df.report_month == df.cur_month, 0, np.where(
        df.report_month == df.next_1_month, 0, np.where(
            df.report_month == df.next_2_month, 0, np.where(
                df.report_month.isnull(), 0, 1
            )
        )
    )
)
df['cur_month'] = np.where(df.cur_month == df.report_month, 1, 0)
df['next_1_month'] = np.where(df.next_1_month == df.report_month, 1, 0)
df['next_2_month'] = np.where(df.next_2_month == df.report_month, 1, 0)

In [28]:
df['assigned'] = np.where(
    (df.cur_month == 0) &
    (df.next_1_month == 0) &
    (df.next_2_month == 0) &
    (df.next_plus_month == 0) &
    (df.staffing_status_final == 'Assigned'), 1, 0
)

df['cancelled'] = np.where(
    (df.cur_month == 0) &
    (df.next_1_month == 0) &
    (df.next_2_month == 0) &
    (df.next_plus_month == 0) &
    (df.staffing_status_final == 'Closed'), 1, 0
)

df['Terminated'] = np.where(
    (df.cur_month == 0) &
    (df.next_1_month == 0) &
    (df.next_2_month == 0) &
    (df.next_plus_month == 0) &
    (df.staffing_status_final == 'Terminated'), 1, 0
)

df['In_Progress'] = np.where(
    (df.cur_month == 0) &
    (df.next_1_month == 0) &
    (df.next_2_month == 0) &
    (df.next_plus_month == 0) &
    (df.staffing_status_final == 'In Progress'), 1, 0
)

df['Booked'] = np.where(
    (df.cur_month == 0) &
    (df.next_1_month == 0) &
    (df.next_2_month == 0) &
    (df.next_plus_month == 0) &
    (df.staffing_status_final == 'Booked'), 1, 0
)

df['Proposed'] = np.where(
    (df.cur_month == 0) &
    (df.next_1_month == 0) &
    (df.next_2_month == 0) &
    (df.next_plus_month == 0) &
    (df.staffing_status_final == 'Proposed'), 1, 0
)

df['Not_Actionable'] = np.where(
    (df.cur_month == 0) &
    (df.next_1_month == 0) &
    (df.next_2_month == 0) &
    (df.next_plus_month == 0) &
    (df.staffing_status_final == 'Not Actionable'), 1, 0
)

df['Onboarding'] = np.where(
    (df.cur_month == 0) &
    (df.next_1_month == 0) &
    (df.next_2_month == 0) &
    (df.next_plus_month == 0) &
    (df.staffing_status_final == 'Onboarding'), 1, 0
)

df['On_Hold'] = np.where(
    (df.cur_month == 0) &
    (df.next_1_month == 0) &
    (df.next_2_month == 0) &
    (df.next_plus_month == 0) &
    (df.staffing_status_final == 'On Hold'), 1, 0
)

df['Open'] = np.where(
    (df.cur_month == 0) &
    (df.next_1_month == 0) &
    (df.next_2_month == 0) &
    (df.next_plus_month == 0) &
    (df.staffing_status_final == 'Open'), 1, 0
)

In [32]:
df['total'] = 1

In [33]:
df

,date,bt,position_id,assignee_id,account_manager_id,actual_start_date,actual_end_date,created,creator_id,last_assigned_date,...,cancelled,Terminated,In_Progress,Booked,Proposed,Not_Actionable,Onboarding,On_Hold,Open,total
0,2019-03-28,"DateTimeRange(datetime.datetime(2019, 3, 23, 5...",131759001,4060741400317351406,4060741400007668548,2019-03-22,None,2019-03-06 14:14:44.201,4060741400006197220,2019-03-22 14:59:58.916000,...,0,0,0,0,0,0,0,0,0,1
1,2019-03-28,"DateTimeRange(datetime.datetime(2019, 3, 23, 6...",127552254,4060741400035500236,4060741400354842536,2019-02-01,None,2019-01-10 10:19:29.369,4000741400012256570,2019-01-22 13:03:35.191000,...,0,0,0,0,0,0,0,0,0,1
2,2019-03-28,"DateTimeRange(datetime.datetime(2019, 3, 23, 9...",130060306,4060741400343637295,4000741400006908159,2019-04-16,None,2019-02-13 08:59:30.908,4000741400002940755,2019-02-26 12:31:02.255000,...,0,0,0,0,0,0,0,0,0,1
3,2019-03-28,"DateTimeRange(datetime.datetime(2019, 3, 23, 7...",126803401,4060741400041080245,4000741400005686069,2019-01-23,None,2018-12-26 13:15:35.763,4000741400000896149,2019-01-23 11:00:08.526000,...,0,0,0,0,0,0,0,0,0,1
4,2019-03-28,"DateTimeRange(datetime.datetime(2019, 3, 26, 1...",77415585,4000741400003244865,4000741400006444598,2017-09-27,None,2017-09-27 10:46:43.234,4060741400047870320,2017-09-27 11:04:38.941000,...,0,0,0,0,0,0,0,0,0,1
5,2019-03-28,"DateTimeRange(datetime.datetime(2019, 3, 24, 6...",130102896,4060741400051307021,4060741400006949008,2019-03-01,None,2019-02-13 14:12:34.459,4060741400007726854,2019-02-26 17:09:10.349000,...,0,0,0,0,0,0,0,0,0,1
6,2019-03-28,"DateTimeRange(datetime.datetime(2019, 3, 25, 0...",132268156,4060741400346089215,4060741400037533145,2019-04-01,None,2019-03-14 10:05:29.694,4060741400008624001,2019-03-22 12:41:12.045000,...,0,0,0,0,0,0,0,0,0,1
7,2019-03-28,"DateTimeRange(datetime.datetime(2019, 3, 27, 1...",133455766,4060741400042497249,4060741400006949008,2019-04-01,None,2019-03-27 14:16:07.381,4060741400007726854,2019-03-27 14:25:54.001000,...,0,0,0,0,0,0,0,0,0,1
8,2019-03-28,"DateTimeRange(datetime.datetime(2019, 3, 26, 1...",130588435,4060741400354328245,4000741400006444598,2019-04-01,None,2019-02-19 10:36:06.845,4060741400047870320,2019-02-19 11:08:43.593000,...,0,1,0,0,0,0,0,0,0,1
9,2019-03-28,"DateTimeRange(datetime.datetime(2019, 3, 25, 1...",133248680,None,4060741400008593864,None,None,2019-03-25 12:57:57.804,4060741400311489240,NaT,...,1,0,0,0,0,0,0,0,0,1


In [38]:
df.date = df.date.astype(str)

In [40]:
pd.pivot_table(df, index='date', values=['total', 'cur_month', 'next_1_month', 'next_2_month', 'next_plus_month',
    'assigned', 'Terminated', 'cancelled', 'Open', 'In_Progress', 'Proposed', 'Booked',
    'Onboarding', 'Not_Actionable', 'On_Hold'], aggfunc=np.sum)[
    ['total', 'cur_month', 'next_1_month', 'next_2_month', 'next_plus_month',
    'assigned', 'Terminated', 'cancelled', 'Open', 'In_Progress', 'Proposed', 'Booked',
    'Onboarding', 'Not_Actionable', 'On_Hold']
]#.to_excel('../data/billable_pos_pipeline.xlsx')

,total,cur_month,next_1_month,next_2_month,next_plus_month,assigned,Terminated,cancelled,Open,In_Progress,Proposed,Booked,Onboarding,Not_Actionable,On_Hold
date,,,,,,,,,,,,,,,
2017-01-01,4222,1113,352,132,263,72,1407,862,0,0,0,0,0,0,21
2017-01-07,3857,667,407,136,279,73,1379,895,0,0,0,0,0,0,21
2017-01-14,3814,398,491,168,309,73,1395,959,0,0,0,0,0,0,21
2017-01-21,3732,189,609,201,351,73,1326,968,0,0,0,0,0,0,15
2017-01-28,4140,79,797,299,458,73,1414,1005,0,0,0,0,0,0,15
2017-02-01,3866,511,342,160,338,73,1421,1006,0,0,0,0,0,0,15
2017-02-07,3787,364,408,184,359,73,1368,1016,0,0,0,0,0,0,15
2017-02-14,3816,284,501,221,394,73,1321,1006,0,0,0,0,0,0,16
2017-02-21,3873,129,641,261,437,73,1305,1011,0,0,0,0,0,0,16


In [49]:
df[
    (df.date == '2019-01-01') &
    (df.Open == 1)
][['position_name', 'planned_start_date', 'planned_end_date', 'container_name', 'customer_name']]

,position_name,planned_start_date,planned_end_date,container_name,customer_name
74602,None,2019-04-30,2019-12-31,RELS-RMCQ,Elsevier
74603,None,2019-04-30,2019-12-31,RELS-RMCQ,Elsevier
74608,None,2019-04-30,2019-12-31,RELS-RMCQ,Elsevier
74612,None,2019-01-31,2019-01-31,RELS-RMCQ,Elsevier
74616,None,2019-04-30,2019-12-31,RELS-RMCQ,Elsevier
74927,None,2017-05-01,2017-12-31,TCO-EMC,Tengizchevroil LLP
77616,None,2018-12-10,2019-01-09,CCCA-NRGP,Coca Cola Company APAC
79167,Java integration - Kafka,2018-07-02,2018-08-24,TRUN-ARCH,Trunomi
79173,None,2019-01-15,2019-12-31,LIBG-WIFI,Liberty Global Services
79466,None,2019-01-15,2019-12-31,LIBG-WIFI,Liberty Global Services


In [56]:
df = pd.read_pickle('../data/df_revenue_model_training_2019-03-31_old.pkl')

In [2]:
countries_dict = loc_ids.id_city_country.drop_duplicates('country_id').set_index('country_id').country.to_dict()

bad_calendars = {
    'Canada' : 'Canada, Calgary',
    'Switzerland' : 'Switzerland, Zurich',
    'India' : 'India, Hyderabad',
    'Spain' : 'Spain, Malaga',
    'Australia' : 'Australia, New South Wales',
    'Hong Kong SAR' : 'Hong Kong',
    'China- Not Active' : 'China'
}

In [4]:
day_start = datetime.datetime.strptime('2014-01-01', '%Y-%m-%d')
day_end = datetime.datetime.today()

days = pd.date_range(day_start, day_end)
calendar = pd.DataFrame({'day': days})
calendar['day'] = calendar['day'].dt.date

In [6]:
for c in sorted(countries_dict.values()):
    
    logger.info(c)
    country_id = [k for k,v in countries_dict.items() if v==c][0]
    
    if c in ['Israel', 'Portugal', 'Viet Nam']: #no calendar updates for this countries
        c_cal = pd.DataFrame({'day' : pd.bdate_range(start=day_start, end=day_end)})
        c_cal[country_id] = 1
        
    else:

        if c in bad_calendars.keys():        
            cal_id_df = postgres.get_data('get_country_calendar_id.sql', param = bad_calendars[c], param_type='day')

        else:    
            cal_id_df = postgres.get_data('get_country_calendar_id.sql', param = c, param_type='day')

        try:
            c_id = cal_id_df['calendar_id'][0]
        except:
            logger.debug('problem with country calendar for {}'.format(c))

        c_cal = postgres.get_data('business_days_by_country_id.sql', param = c_id, param_type='day')
        c_cal[country_id] = 1
        c_cal = c_cal.drop_duplicates()
        
    c_cal['day'] = pd.to_datetime(c_cal['day']).dt.date
    
    calendar = pd.merge(calendar, c_cal,
      on = 'day',
      how = 'left')
    
    calendar = calendar.fillna(0)
    calendar[country_id] = calendar[country_id].astype(int)
    
calendar['day'] = calendar['day'].astype(str)
calendar = calendar.set_index('day')

2019-10-18 11:27:26,430 - root : INFO : Armenia
2019-10-18 11:27:27,707 - root : INFO : Australia
2019-10-18 11:27:28,714 - root : INFO : Austria
2019-10-18 11:27:29,763 - root : INFO : Belarus
2019-10-18 11:27:30,911 - root : INFO : Belgium
2019-10-18 11:27:31,878 - root : INFO : Bulgaria
2019-10-18 11:27:32,901 - root : INFO : Canada
2019-10-18 11:27:33,858 - root : INFO : China
2019-10-18 11:27:34,982 - root : INFO : China- Not Active
2019-10-18 11:27:35,993 - root : INFO : Cyprus
2019-10-18 11:27:36,976 - root : INFO : Czech Republic
2019-10-18 11:27:38,020 - root : INFO : Denmark
2019-10-18 11:27:39,019 - root : INFO : France
2019-10-18 11:27:40,236 - root : INFO : Germany
2019-10-18 11:27:41,394 - root : INFO : Hong Kong SAR
2019-10-18 11:27:42,387 - root : INFO : Hungary
2019-10-18 11:27:43,484 - root : INFO : India
2019-10-18 11:27:44,507 - root : INFO : Ireland
2019-10-18 11:27:45,555 - root : INFO : Israel
2019-10-18 11:27:45,605 - root : INFO : Italy
2019-10-18 11:27:46,637 

In [7]:
calendar

,4000741400000756803,4060741400008501992,4060741400034686867,4000602900000005345,4060741400304650362,4060741400008679559,4000741400000172696,4060741400007779857,4000741400002544657,4060741400035398530,...,4060741400034895894,4000741400007028830,4000741400009107797,4060741400340599192,4060741400037597123,4000602900000005341,4000602900000005338,4000741334650021875,8150000000000000024,4060741400349324343
day,,,,,,,,,,,,,,,,,,,,,
2014-01-01,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2014-01-02,0,0,1,0,0,0,0,1,1,0,...,0,1,0,0,0,1,1,1,0,1
2014-01-03,0,0,1,1,0,1,0,1,1,0,...,0,1,1,0,0,1,1,1,0,1
2014-01-04,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-01-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-01-06,0,0,1,0,0,1,0,1,1,0,...,0,0,1,0,0,1,1,1,0,1
2014-01-07,0,0,1,0,0,1,0,1,1,0,...,0,1,1,0,0,1,1,0,0,1
2014-01-08,1,0,1,1,0,1,0,1,1,0,...,0,1,1,0,0,1,1,1,0,1
2014-01-09,1,0,1,1,0,1,0,1,1,0,...,0,1,1,0,0,1,1,1,0,1


In [83]:
df[['date', 'workload_start_date', 'staffing_locations_country_ids']]

,date,workload_start_date,staffing_locations_country_ids
1325497932019-03-28,2019-03-28,2019-04-01,"[4060741400304650362, 4060741400007779857, 406..."
1325475142019-03-28,2019-03-28,2019-04-15,"[4060741400304650362, 4060741400007779857, 406..."
1325472732019-03-28,2019-03-28,2019-04-15,"[4060741400304650362, 4060741400007779857, 406..."
1325469922019-03-28,2019-03-28,2019-04-15,"[4060741400304650362, 4060741400007779857, 406..."
1325471862019-03-28,2019-03-28,2019-04-15,"[4060741400304650362, 4060741400007779857, 406..."
1325462092019-03-28,2019-03-28,2019-04-15,"[4060741400304650362, 4060741400007779857, 406..."
1325469952019-03-28,2019-03-28,2019-04-15,"[4060741400304650362, 4060741400007779857, 406..."
1325462122019-03-28,2019-03-28,2019-04-15,"[4060741400304650362, 4060741400007779857, 406..."
1322254592019-03-28,2019-03-28,2019-04-29,"[4060741400304650362, 4060741400007779857, 406..."
1325473302019-03-28,2019-03-28,2019-04-15,"[4060741400304650362, 4060741400007779857, 406..."


In [8]:
import termination
reload(termination)

<module 'termination' from '../src\termination.pyc'>

In [13]:
d = '2019-09-23'

lst = []

d = datetime.datetime.strptime(d, '%Y-%m-%d')

while d <= datetime.datetime.today():
    lst.append(datetime.datetime.strftime(d, '%Y-%m-%d'))
    d = d + relativedelta(days=1)

In [72]:
tm = termination.Termination(job='making_predictions', date=['2019-09-22', '2019-09-23'])

2019-10-18 14:28:32,625 - is_billable : INFO : is_billablemaking_predictions_2019-09 dataset creation query started
2019-10-18 14:34:14,385 - is_billable : INFO : is_billablemaking_predictions_2019-09 dataset was created successfully


In [147]:
tm.df[
    (tm.df.date == datetime.date(2019, 9, 23))
][['date', 'position_id', 'staffing_locations']][400:450]

,date,position_id,staffing_locations
6210,2019-09-23,146020476,"[{""id"": ""8150000000000067776"", ""name"": ""Krasno..."
6211,2019-09-23,146537189,"[{""id"": ""8150000000000067776"", ""name"": ""Krasno..."
6212,2019-09-23,146021544,"[{""id"": ""8150000000000067776"", ""name"": ""Krasno..."
6213,2019-09-23,146463479,"[{""id"": ""8150000000000067776"", ""name"": ""Krasno..."
6214,2019-09-23,146016740,"[{""id"": ""8150000000000067776"", ""name"": ""Krasno..."
6215,2019-09-23,146020043,"[{""id"": ""8150000000000067776"", ""name"": ""Krasno..."
6216,2019-09-23,147122240,"[{""id"": ""8150000000000067776"", ""name"": ""Krasno..."
6217,2019-09-23,143934215,"[{""id"": ""4060741400005350099"", ""name"": ""New Yo..."
6218,2019-09-23,147043906,"[{""id"": ""4000610700000001042"", ""name"": ""Saint-..."
6219,2019-09-23,140702856,"[{""id"": ""4000610700000001042"", ""name"": ""Saint-..."


In [119]:
import auxiliary as aux
reload (aux)

loc_ids = aux.City_Country()

In [120]:
tmp = tm.df.copy()

In [121]:
tmp = loc_ids.city_country_ids_processor(tmp)

../src\auxiliary.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df = df[df.is_any_location != '-1']
../src\auxiliary.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
../src\auxiliary.py:183: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df = pd.concat([any_loc, df])


In [122]:
tmp[['position_id', 'staffing_locations_city_ids', 'staffing_locations_country_ids']]

,position_id,staffing_locations_city_ids,staffing_locations_country_ids
33,147423496,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."
226,127033271,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."
228,127033163,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."
446,125351429,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."
447,125351470,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."
720,134403341,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."
721,134403117,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."
878,140834639,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."
880,140841199,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."
1603,145645781,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."


In [148]:
tmp[
    (tmp.position_id == 147797010)
]

,date,bt,position_id,assignee_id,account_manager_id,actual_start_date,actual_end_date,created,creator_id,last_assigned_date,...,last_staffing_status_update_date,supply_owners,seniority_level,demand_owners,pmc_roles,gbu_id,customer_name,timesheet_date,staffing_locations_city_ids,staffing_locations_country_ids
4290,2019-09-22,"DateTimeRange(datetime.datetime(2019, 9, 20, 1...",147797010,None,4000741400013246452,None,None,2019-09-19 07:41:49.756,4060741400039942278,NaT,...,2019-09-19 07:55:14.859,[],"[{""id"": ""0"", ""name"": ""A1"", ""_class"": ""SimpleEn...","[{""id"": ""4060741400047322959"", ""name"": ""Ivan D...",[],BD18BECA-2ED3-E611-80CB-001DD8B71DB6,Brightcove Inc.,None,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."
6249,2019-09-23,"DateTimeRange(datetime.datetime(2019, 9, 22, 7...",147797010,None,4000741400013246452,None,None,2019-09-19 07:41:49.756,4060741400039942278,NaT,...,2019-09-19 07:55:14.859,[],"[{""id"": ""0"", ""name"": ""A1"", ""_class"": ""SimpleEn...","[{""id"": ""4060741400047322959"", ""name"": ""Ivan D...",[],BD18BECA-2ED3-E611-80CB-001DD8B71DB6,Brightcove Inc.,None,"[4060741400304942841, 4060741400039759369, 406...","[4060741400304650362, 4060741400007779857, 406..."


In [53]:
a

'[{"id": "-1", "name": "Any Location", "type": "City", "_class": "PositionLocation", "parent": null, "preferred": false}]'

In [55]:
a.split('", "name": "')[0]

'[{"id": "-1'

In [62]:
[
    loc.split('", "name": "')[0] for loc in [
        a for a in [
            k if '"parent"' in k else '' for k in st.split('"id": "')[1:]
        ] if a != ''
    ]
]

['-1']

In [60]:
[loc.split('", "name": "')[0] for loc in st.split('"id": "')[1:]]

['-1']

In [58]:
st.split('"id": "')[1:]

['-1", "name": "Any Location", "type": "City", "_class": "PositionLocation", "parent": null, "preferred": false}]']

In [74]:
x1 = '[{"id": "4060741400304650362", "name": "Belgium", "_class": "PositionLocation", "preferred": false}, {"id": "4000741400000172696", "name": "Canada", "_class": "PositionLocation", "preferred": false}, {"id": "600100000009448", "name": "Grodno", "_class": "PositionLocation", "preferred": false}]'

In [75]:
x2 = '[{"id": "600100000009450", "name": "Moscow", "_class": "PositionLocation", "preferred": false}]'

In [76]:
x3 = '[{"id": "4000613900000001955", "name": "Saratov", "type": "City", "_class": "PositionLocation", "parent": {"id": "4000602900000005343", "name": "Russia"}, "preferred": false}]'

['4000613900000001955']

In [112]:
str([
    a for a in [
        k if '"parent"' in k else '' for k in x1.split('"id": "')[1:]
    ] if a != ''
])

'[]'